Importando libraries

In [60]:
import pandas as pd
import numpy as np
import talib
from funcoes_preco import organize_data, get_efficiency_ratio, get_stats
import dwx_query
from datetime import datetime, timedelta
from bisect import bisect_right

Dados de entrada

In [61]:
files_dir = 'C:/Users/guicr/AppData/Roaming/MetaQuotes/Terminal/'\
            'D0E8209F77C8CF37AD8BF550E51FF075/MQL5/Files/'
symbol = 'Bra50Feb22'
timeframes = ['M5', 'M15']
start = datetime(2020, 1, 1).timestamp()
end = datetime(2020, 6, 30).timestamp()

Gravando dados recebidos em um dicionário

In [62]:
data = {timeframe: dwx_query.inform_historic_data(files_dir, symbol, timeframe, 
                                                  start, end) 
        for timeframe in timeframes}

Bra50Feb22_M5
historic_data: Bra50Feb22 M5 12761 bars
Bra50Feb22_M15
historic_data: Bra50Feb22 M15 4319 bars


Enviando os dados para dataframes

In [63]:
symbol_data = {timeframe: pd.DataFrame(data[timeframe], 
                                       columns=['Date', 'Open', 'High', 'Low', 
                                                'Close', 'Tick Volume', 
                                                'Real Volume'])
               for timeframe in timeframes}

Copiando os dataframes para uma variável de manipulação

In [64]:
eqty_table = symbol_data.copy()

Calculando o retorno barra a barra

In [65]:
pct_change = {timeframe: eqty_table[timeframe]['Close'].pct_change() 
              for timeframe in timeframes}
for timeframe in timeframes:
    eqty_table[timeframe]['Percent Change'] = pct_change[timeframe]

Triggers

In [66]:
cci_period = 20
cci = talib.CCI(eqty_table['M5']['High'], eqty_table['M5']['Low'], 
                eqty_table['M5']['Close'], timeperiod=cci_period)
eqty_table['M5']['CCI'] = cci

In [67]:
std_dev_period = 15
std_dev = get_stats(eqty_table['M5']['Close'], std_dev_period)['std']
eqty_table['M5']['Standard Deviation'] = std_dev

Filters

In [68]:
er_period = 5
efficiency_ratio = get_efficiency_ratio(eqty_table['M15']['Close'], 
                                        timeperiod=er_period)
eqty_table['M15']['Efficiency Ratio'] = efficiency_ratio['efficiency_ratio']


In [69]:
skew_period = 15
skewness = get_stats(eqty_table['M15']['Close'], skew_period)['skewness']
eqty_table['M15']['Skewness'] = skewness

Preparação do Dataframe para receber as informações de trade.
Integração entre o Timeframe maior com o menor.

Mudar o tipo da coluna 'Date' para datetime.

In [70]:
for timeframe in timeframes:
    eqty_table[timeframe]['Date'] = pd.to_datetime(eqty_table[timeframe]['Date'])

Colocando um marcador para identificar a correspondência entre o Timeframe 
maior e o menor.

In [71]:
lwr_timeframe_table = eqty_table['M5'].copy()
hgr_timeframe_table = eqty_table['M15'].copy()

In [72]:
index_higher_timeframe = []
for row_lwr_timeframe in lwr_timeframe_table.index:
    index = bisect_right(hgr_timeframe_table['Date'], 
                         lwr_timeframe_table.loc[row_lwr_timeframe, 'Date'])
    index_higher_timeframe.append(index-1)

In [73]:
eqty_table['M5']['Index Higher Timeframe'] = index_higher_timeframe

Configuração dos sinais de compra e venda.

In [101]:
open_position = False
position = []
pos_exit = []
pos_type = []

for row in eqty_table['M5'].index:
    
    if row < cci_period:
        continue
    
    # Sinal entrada de Compra
    elif not open_position and \
        eqty_table['M5'].loc[row-1, 'CCI'] < -100 and \
        eqty_table['M5'].loc[row, 'CCI'] > -100 and \
        eqty_table['M15'].loc[eqty_table['M5']['Index Higher Timeframe'][row]-1, 
                                'Skewness'] < 0 and \
        eqty_table['M15'].loc[eqty_table['M5']['Index Higher Timeframe'][row]-1,
                              'Efficiency Ratio'] < 0.5:
            open_position = True
            pos_type = 'Buy'
            std = eqty_table['M5'].loc[row, 'Standard Deviation']
            position.append([row, pos_type, eqty_table['M5'].loc[row+1, 'Open'],
                             std])
    
    # Sinal de Saída da Compra
    elif open_position and \
        pos_type == 'Buy' and \
        eqty_table['M5'].loc[row-1, 'CCI'] < 100 and \
        eqty_table['M5'].loc[row, 'CCI'] > 100:
            open_position = False
            pos_type = 'Exit Buy'
            position.append([row, pos_type, eqty_table['M5'].loc[row, 'Close'],
                             0])
    
    elif open_position and \
        pos_type == 'Buy' and \
        eqty_table['M5'].loc[row-1, 'CCI'] > -100 and \
        eqty_table['M5'].loc[row, 'CCI'] < -100:
            open_position = False
            pos_type = 'Exit Buy'
            position.append([row, pos_type, eqty_table['M5'].loc[row, 'Close'],
                             0])
            
    # Sinal entrada de Venda
    elif not open_position and \
        eqty_table['M5'].loc[row-1, 'CCI'] > 100 and \
        eqty_table['M5'].loc[row, 'CCI'] < 100 and \
        eqty_table['M15'].loc[eqty_table['M5']['Index Higher Timeframe'][row]-1, 
                                'Skewness'] > 0 and \
        eqty_table['M15'].loc[eqty_table['M5']['Index Higher Timeframe'][row]-1,
                              'Efficiency Ratio'] < 0.5:
            open_position = True
            pos_type = 'Sell'
            std = eqty_table['M5'].loc[row, 'Standard Deviation']
            position.append([row, pos_type, eqty_table['M5'].loc[row+1, 'Open'],
                             std])
            
    # Sinal de Saída da Venda
    elif open_position and \
        pos_type == 'Sell' and \
        eqty_table['M5'].loc[row-1, 'CCI'] > -100 and \
        eqty_table['M5'].loc[row, 'CCI'] < -100:
            open_position = False
            pos_type = 'Exit Sell'
            position.append([row, pos_type, eqty_table['M5'].loc[row, 'Close'],
                             0])
    
    elif open_position and \
        pos_type == 'Sell' and \
        eqty_table['M5'].loc[row-1, 'CCI'] < 100 and \
        eqty_table['M5'].loc[row, 'CCI'] > 100:
            open_position = False
            pos_type = 'Exit Sell'
            position.append([row, pos_type, eqty_table['M5'].loc[row, 'Close'],
                             0])

In [159]:
position[0:15]

[[49, 'Sell', 118300.0, 90.91754506144565],
 [50, 'Exit Sell', 118390.0, 0],
 [95, 'Sell', 118700.0, 142.29624809608376],
 [103, 'Exit Sell', 118895.0, 0],
 [153, 'Buy', 118495.0, 135.37848179579106],
 [155, 'Exit Buy', 118160.0, 0],
 [158, 'Buy', 118325.0, 128.64680330268607],
 [166, 'Exit Buy', 118660.0, 0],
 [234, 'Buy', 117140.0, 172.45740214776387],
 [242, 'Exit Buy', 116955.0, 0],
 [255, 'Sell', 117290.0, 196.26908965895663],
 [256, 'Exit Sell', 117530.0, 0],
 [374, 'Sell', 116745.0, 89.04680417248747],
 [383, 'Exit Sell', 116795.0, 0],
 [514, 'Buy', 116745.0, 221.29768789272666]]

In [172]:
df = pd.DataFrame(position, columns=['Index', 'Position Type', 'Position', 'Standard Deviation'])

In [173]:
df

,Index,Position Type,Position,Standard Deviation
0,49,Sell,118300.0,90.917545
1,50,Exit Sell,118390.0,0.000000
2,95,Sell,118700.0,142.296248
3,103,Exit Sell,118895.0,0.000000
4,153,Buy,118495.0,135.378482
...,...,...,...,...
471,12674,Exit Sell,94420.0,0.000000
472,12678,Buy,94600.0,218.233820
473,12680,Exit Buy,94940.0,0.000000
474,12704,Buy,94625.0,92.235207


In [174]:
df2 = df.iloc[::2] 
df2.reset_index(inplace=True)
df2.drop('index', axis=1, inplace=True)
df2

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Index,Position Type,Position,Standard Deviation
0,49,Sell,118300.0,90.917545
1,95,Sell,118700.0,142.296248
2,153,Buy,118495.0,135.378482
3,158,Buy,118325.0,128.646803
4,234,Buy,117140.0,172.457402
...,...,...,...,...
233,12572,Buy,95405.0,247.328841
234,12621,Buy,94370.0,201.754526
235,12672,Sell,94570.0,141.539158
236,12678,Buy,94600.0,218.233820


In [175]:
df3 = df.iloc[1::2]
df3.reset_index(inplace=True)
df3.drop('index', axis=1, inplace=True)
df3.rename(columns={'Position': 'Position Exit', 
                    'Position Type': 'Position Type Exit',
                    'Standard Deviation': 'Standard Deviation Exit'}, 
           inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [176]:
df = pd.concat([df2, df3], axis=1)
df

,Index,Position Type,Position,Standard Deviation,Index,Position Type Exit,Position Exit,Standard Deviation Exit
0,49,Sell,118300.0,90.917545,50,Exit Sell,118390.0,0.0
1,95,Sell,118700.0,142.296248,103,Exit Sell,118895.0,0.0
2,153,Buy,118495.0,135.378482,155,Exit Buy,118160.0,0.0
3,158,Buy,118325.0,128.646803,166,Exit Buy,118660.0,0.0
4,234,Buy,117140.0,172.457402,242,Exit Buy,116955.0,0.0
...,...,...,...,...,...,...,...,...
233,12572,Buy,95405.0,247.328841,12573,Exit Buy,95360.0,0.0
234,12621,Buy,94370.0,201.754526,12625,Exit Buy,94240.0,0.0
235,12672,Sell,94570.0,141.539158,12674,Exit Sell,94420.0,0.0
236,12678,Buy,94600.0,218.233820,12680,Exit Buy,94940.0,0.0


In [177]:
calc = df['Position'] - df['Position Exit']
calc

0      -90.0
1     -195.0
2      335.0
3     -335.0
4      185.0
       ...  
233     45.0
234    130.0
235    150.0
236   -340.0
237   -300.0
Length: 238, dtype: float64

In [178]:
df.loc[0, 'Position Type']

'Sell'

In [179]:
points = []
for row in df.index:
    if df.loc[row, 'Position Type'] == 'Buy':
        points = np.append(points, df.loc[row, 'Position'] - df.loc[row, 'Position Exit'])
    else:
        points = np.append(points, [df.loc[row, 'Position Exit'] - df.loc[row, 'Position']])

points

array([ 9.000e+01,  1.950e+02,  3.350e+02, -3.350e+02,  1.850e+02,
        2.400e+02,  5.000e+01, -2.700e+02,  4.550e+02, -2.050e+02,
        2.600e+02, -7.500e+01, -2.950e+02, -2.500e+02,  7.000e+01,
        6.000e+01,  7.500e+01,  6.500e+01, -2.250e+02, -5.000e+02,
        4.000e+01, -1.900e+02, -4.700e+02, -1.500e+02,  1.300e+02,
       -7.500e+01,  7.500e+01,  1.050e+02,  1.200e+02,  9.500e+01,
        7.000e+01, -1.800e+02, -3.400e+02,  1.500e+02,  4.000e+01,
        1.700e+02, -1.100e+02, -1.500e+02, -2.750e+02,  1.500e+02,
        1.650e+02,  5.500e+01,  2.300e+02,  1.200e+02,  1.000e+02,
       -4.650e+02,  1.750e+02, -3.850e+02, -9.000e+01, -2.650e+02,
        2.700e+02,  2.800e+02,  3.400e+02, -5.000e+00,  3.050e+02,
       -3.800e+02, -2.150e+02,  1.350e+02,  1.150e+03,  1.400e+02,
        5.450e+02, -2.450e+02,  3.550e+02,  1.950e+02,  2.200e+02,
       -2.200e+02,  1.800e+02, -5.800e+02,  1.400e+02, -1.350e+02,
       -1.150e+02, -3.800e+02,  1.900e+02, -1.750e+02, -5.750e

In [170]:
stds = df['Standard Deviation'] * 2
stds

0      181.835090
1      284.592496
2      270.756964
3      257.293607
4      344.914804
          ...    
233    494.657682
234    403.509053
235    283.078317
236    436.467639
237    184.470413
Name: Standard Deviation, Length: 238, dtype: float64

In [180]:
r = points / stds

r

0      0.494954
1      0.685190
2      1.237272
3     -1.302014
4      0.536364
         ...   
233    0.090972
234    0.322174
235   -0.529889
236   -0.778981
237   -1.626277
Name: Standard Deviation, Length: 238, dtype: float64

In [181]:
sum(r)

-44.8586333989949